Họ tên: Phan Lộc Sơn

MSSV: 19120033

# HW4: Song song hóa Radix Sort

Với các GPU tương đối mới thì để biên dịch chỉ cần dùng câu lệnh: \
`nvcc tên-file.cu -o tên-file-chạy`

Nhưng trên Colab mình thường lấy được GPU khá cũ là Tesla K80 với compute capability (phiên bản phần cứng) là 3.7; để biên dịch đúng với GPU khá cũ này thì bạn cần dùng câu lệnh: \
`nvcc -arch=sm_37 tên-file.cu -o tên-file-chạy` \
Trong đó, 37 chính là compute capability của GPU Tesla K80.

Để phòng trường hợp khi làm bài bạn lấy được GPU có compute capability x.x nhưng khi chấm bài Thầy lại lấy được GPU có compute capability khác x.x, dưới đây mình sẽ có đoạn code Python để tự động lấy 2 con số ứng với compute capability của GPU và lưu vào 2 biến `major` và `minor`:


In [1]:
from numba import cuda
major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

GPU compute capability: 7.5


Một khi đã chạy đoạn code Python ở trên, để biên dịch thì bạn sẽ dùng câu lệnh: \
`nvcc -arch=sm_{major}{minor} tên-file.cu -o tên-file-chạy`

Dưới đây, khi làm bài thì bạn có thể tùy ý thêm/xóa cell. Đừng xóa mấy cell có chữ của Thầy là được.

In [17]:
!nvcc -arch=sm_{major}{minor} HW4.cu -o HW4

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 9345.183 ms

Radix Sort by device
Time: 9148.597 ms
CORRECT :)


In [21]:
!./HW4 256

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 9224.513 ms

Radix Sort by device
Time: 10091.342 ms
CORRECT :)


In [19]:
!./HW4 512

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 9225.333 ms

Radix Sort by device
Time: 9694.218 ms
CORRECT :)


In [20]:
!./HW4 1024

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************

Input size: 16777217

Radix Sort by host
Time: 9307.771 ms

Radix Sort by device
Time: 8837.839 ms
CORRECT :)


Tốc độ của host ở cả 3 lần chạy gần như nhau.
Tốc độ của kernel ở 3 lần chạy có cải thiện khi tăng blocksize lên.

Sự cải thiện này có thể giải thích như sau:

Ở mỗi lần scan, device scan cục bộ các block song song, sau đó, scan tổng của các biến thì lần lượt.

- 1 block scan được nhiều phần tử, mà mỗi phần tử nằm trong shared_mem, nhanh -> giảm thời gian truy xuất

- Giảm số phân tử trong aux array phải scan lần lượt, các phần tử này nằm ở global mem -> tăng tốc độ scan aux array.

- Khi tăng blocksize: ví dụ từ 256 -> 512, ở reduce phase và Post-reduction phase, với 512 phần tử: 1 block 512 scan cục bộ mất 9 + 8 = 17 bước. 2 block 256 scan cục bộ hết 2x(8+7) = 30 bước. (nếu SM còn dư slot và 2 block này chạy song song thì chỉ mất 15 bước, nếu SM hết slot thì 2 block này chạy lần lượt thì mất 30 bước, do size trong mẫu chạy HW4.cu khá lớn nên coi như là SM luôn trong tình trạng hết slot). Ngoài ra 2 block 256 còn phải truy xuất global mem 2 lần để scan tổng aux.